# Assignment 2: Performance Metrics & First Week Flop/s

This assignment has some questions that you need to answer with text, and some code that you need to write.

You should put all of you textual answers in this notebook: `Insert->Insert Cell Below` to create a new cell below
the question, and `Cell->Cell Type->Markdown` to make it a cell for entering text.

You will test your code on the compute nodes of pace-ice, and that it also where we will evaluate it.
Please complete the text portions when you are logged into a head node working locally, and leave the compute nodes for when you actually need them.

**Due: Tuesday, September 2, 9:30 am**

**Total: 10 pts + 2 bonus pts (1 for working on a node with GPUs, 1 for optimizing the flop/s code)**

## Performance Metrics

In class we talked about the _strong-scaling efficiency_ of a parallel algorithm / machine pair: $H_f(P) = T_f(1) / (P T_f(P))$.

We then talked about the _weak-scaling efficiency_ of algorithm $f$ that can be applied to different problem sizes $N$: $E_f(N,P) = T_f(N/P,1) / T_f(N,P)$.

The question came up of how they are related to each other.

First, the notion of strong scaling doesn't have a concept of problem size, so let's add it: let's define

$$H_f(N,P) = T_f(N,1) / (P T_f(N,P)).$$

This is simply strong-scaling efficiency for each problem instance individually.

**Question 1 (1 pt):** Show that the relative order of strong and weak scaling efficiency (Whether $H_f(N,P) < E_f(N,P)$ or $E_f(N,P) < H_f(N,P)$) can be related to the efficiency of the serial algorithm, that is, whether $T_f(N,1)$ as a function of $N$ exhibits superlinear or sublinear behavior.

Notice that $T_f(N,P)$ appear both in $H_f(N,P)$ and $E_f(N,P)$, thus to compare $H_f(N,P)$ and $E_f(N,P)$ we only need to compare $\frac{T_f(N,1)}{P}$ and $T_f(\frac{N}{P},1)$.

Consider $f(N) = T_f(N,1)$, if $f(N)$ is superlinear, then $$f(N) = f(P\frac{N}{P}) > P f(\frac{N}{P})$$ thus $E_f(N,P) < H_f(N,P)$.

Similarly if $f(N)$ is sublinear, then $f(N) = f(P\frac{N}{P}) < P f(\frac{N}{P})$, then $E_f(N,P) > H_f(N,P)$

## PACE-ICE

**Head node exercise 1 (1 pt):** What command should you run from a head node to see a list of all the compute nodes in `coc-ice` and their availability? [Resource for this question: the [orientation slides](http://pace.gatech.edu/sites/default/files/pace-ice_orientation_2.pdf)]

In [ ]:
# <-- Put your command there
pace-check-queue coc-ice

Try it out: open up this notebook on a head node and compare the list you get to the [orientation slides](http://pace.gatech.edu/sites/default/files/pace-ice_orientation_2.pdf).  You'll see that it has grown, and they haven't updated the orientation slides.  We'll just have to find out what all these new nodes are for ourselves.

---
### A word on running jupyter on pace-ice:

As we discussed in class, screen refresh can be a bit laggy if you try to run a jupyter notebook through a browser opened on the head node or a compute node.  See the [guide](../../notes/logistics/compute-node-notebook.ipynb) in the notes for instructions on runnin the jupyter server on the compute nodes and the browser on your own computer.  You don't have to work directly in the notebook: you can work on you answers in the terminal, and then paste them into the notebook, as long as you're confident that they are correct.

---

**Head node exercise 2 (1 pt):** From the output of the above answer, you can probably see that we have a few different types of nodes to work with.  Fill in the blanks in the list below, describing the _properties_ of the different types. [Command line tools you might want to use: `qnodes`, `grep`, `sort`]

1. **23** nodes with **28** CPU core(s) and no GPUs
2. **12** nodes with **12** CPU core(s) and **2** GPU(s) of type NVIDIA Tesla **K40**.
3. **12** nodes with **8** CPU core(s) and **1** GPU(s) of type NVIDIA Tesla **P100** (this group includes `rich133-s42-21.pace.gatech.edu`, even though that the type of GPU is not listed for this node)
3. One node with **8** CPU core(s) and **2** GPU(s) of type NVIDIA Tesla **P100** `rich133-k33-17.pace.gatech.edu`
5. One node (`rich133-s30-20.pace.gatech.edu`) with **24** CPU core(s) (currently offline).

In [ ]:
qnodes | grep -10 rich133-s30-20.pace.gatech.edu

**Head node exercise 3 (1 pt):** For the next questions, I need you to log in to compute nodes to find out about them, but you need to be able to specify which type of compute nodes you are accessing.

For each of the types of nodes 1, 2, and 3 in the question above, give me a `qsub` command to start a `jupyter_notebook_script.sh` job on that type of node, with the following requirements:

* The job should give you exclusive access to one node and all its cores and devices.
* The job should begin in the CSE6230 directory.
* The job should end after 30 minutes.

[Resources: [compute-node-notebook.ipynb](../../notes/logistics/compute-node-notebook.ipynb), [orientation slides](http://pace.gatech.edu/sites/default/files/pace-ice_orientation_2.pdf)]

In [ ]:
# put the qsub command for type 1 in this cell
qsub -q coc-ice -d $CSE6230_DIR -l nodes=rich133-h35-15-r.pace.gatech.edu:ppn=28,walltime=00:30:00 $CSE6230_DIR/utils/jupyter_notebook_job.sh

In [ ]:
# put the qsub command for type 2 in this cell
qsub -q coc-ice -d $CSE6230_DIR -l nodes=rich133-s30-10.pace.gatech.edu:ppn=12:gpus=2:teslak40:exclusive_process,walltime=00:30:00 $CSE6230_DIR/utils/jupyter_notebook_job.sh

In [ ]:
# put the qsub command for type 3 in this cell
qsub -q coc-ice -d $CSE6230_DIR -l nodes=rich133-k40-18.pace.gatech.edu:ppn=8:gpus=1:teslap100:exclusive_process,walltime=00:30:00 $CSE6230_DIR/utils/jupyter_notebook_job.sh

## What have we got to work with?

Now, we need to switch from a notebook running on the head node to one running on a compue node, so `File->Save and Checkpoint` this notebook and `File->Close and Halt` it.  (Now would also be a good time to `git add` and `git commit` changes to this file.)  Use one of your ineractive job scripts to connect to a compute node and run the notebook there.
See you on the other side!

---

Okay, you're running on the compute node.

**Compute node exercise 1 (2 pt):** Using bash scripting (`awk`, `grep`, `sed`) or any other tool you like (you could, e.g., write a python script in a separate file and call it, as long as you `git add` it), set the variables in the cell below so that the printout that follows is correct.  You script should be correct on any type of compute node.

Resources: the file `/proc/cpuinfo`, the utility `nvidia-smi`; if you are very new to using a shell command line and the utilities that go with it in linux, please look at the [training slides on Linux](https://pace.gatech.edu/training) from PACE.

Note: when you run a command in backticks, you can assign its value to a variable like

```bash
MY_FILES=`ls -al`
```

Also note: when ever you encounter a new program or utility `AAA` that you've never used before, `man AAA` or `AAA --help` are the first places to go if you want to know what different command line flags do.

Some [one-liners](https://en.wikipedia.org/wiki/One-liner_program) that you may find useful:

* `grep -P -m 1 -o -e "(?<=XXX\s: ).*" YYY`: look in file `YYY` for the string "XXX   : " (with an arbitrary number of spaces between `XXX` and `:`) and print what comes after that on the line
* `wc -l YYY` counts the number of lines in file `YYY`
* most command line utilities that read files can also read the output of a previous command with a pipe `|`, for example to count the number of files in a directory:

```bash
ls -al | wc -l
```

* `grep -c "ZZZ" YYY` count the number of times the string `ZZZ` occurs in file `YYY`
* Nodes without GPUs won't have the `nvidia-smi` utility.  You can tell when a utility is unavailable if `which AAA` returns an error.  If you want to write a one-liner that only runs command `AAA` when `nvidia-smi` when it's available, you can do that like this:

```bash
(which nvidia-smi &> /dev/null) && (AAA)
```

In [ ]:
grep core /proc/cpuinfo

In [ ]:
grep "model name" /proc/cpuinfo | head -1 | awk -F: '{print $2}'

In [ ]:
nvidia-smi -L | head -1 | awk '{ print $3" "$4 }'

In [28]:
CPU_NAME=`grep "model name" /proc/cpuinfo | head -1 | awk -F: '{print $2}'`
CORE_COUNT=`grep "core id" /proc/cpuinfo | wc -l`
GPU_NAME=`(which nvidia-smi &> /dev/null) && (nvidia-smi -L | head -1 | awk '{ print $3" "$4 }')`
GPU_COUNT=`(which nvidia-smi &> /dev/null) && (nvidia-smi -L | wc -l)`

: 1

In [29]:
echo "This nodes has ${CORE_COUNT} cores: its architecture is (Manufacturer, Product Id) ${CPU_NAME}"
if [[ ! $GPU_COUNT || $GPU_COUNT == 0 ]] ;  then
    echo "This node has no GPUs"
else
    echo "This node has ${GPU_COUNT} GPUs: its/their architecture is (Manufacturer, Product Id) ${GPU_NAME}"
fi

This nodes has 28 cores: its architecture is (Manufacturer, Product Id)  Intel(R) Xeon(R) CPU E5-2680 v4 @ 2.40GHz
This node has no GPUs


**Compute node exercise 2 (1 pt):** After you have logged out of the compute node, use whatever resources published on the web you can find to estimate the peak _single precision_ (aka FP32) flop/s of this node (you only need to do this step for one of the types of nodes, not all of them).

[Resources: [ark.intel.com](https://ark.intel.com), [wikipedia:FLOPS](https://en.wikipedia.org/wiki/FLOPS), [wikichip](https://en.wikichip.org), our notebook on [processors](../../notes/processors/processors-alone.ipynb)

I have chosen node type **1**

The peak flop/s for this node is **2150.4** gigaflop/s.  Here is how I calculated that:

2.4 GHz * 28 cores * 32(Broadwell)

## Flop/s fever

We've got to scratch that itch: we just want to go fast.  Okay, let's get it out of our system, and we'll look at more practical computations in future assignments.

You should choose one of the node types for this task.  Because this is more complex if multiple devices are involved
**1 bonus point** is earned for choosing a node with GPUs.

**Compute node exercise 3 (2 pts):** The command below will compile and runs essentially the following computation:

```C
for (i = 0; i < Nh; i++) { /* this loop will run on the "host" (CPUs) */
  for (j = 0; j < T; j++) {
    ah[i] = ah[i] * b + c;
  }
}

for (i = 0; i < Nd; i++) { /* this loop will run on the "device" (GPUs) */
  for (j = 0; j < T; j++) {
    ad[i] = ad[i] * b + c;
  }
}
```
And it will report the flop/s for the whole calculation.

`Nh` array entries will be on the host and `Nd` entries will be on each of the devices.  Try to find values of `Nh`, `Nd`, and `T`, and (optionally) compiler optimization flags that give you the highest flop/s.  Things to consider:

- Try to make your whole computation run for about a second.
- The time reported is the maximum time for any device: if one sits idle while the other finishes, it will rob you of flop/s.
- I suggest looking at one type of device at a time: set one of `Nh` or `Nd` to zero.  Once you've found your best flop/s for that device, optimize the other, and then try to strike a balance.
- Experiment with the merits of putting more weight on `Nh` and `Nd` vs more weight on `T`.
  Try to use **Little's Law** to make sure that you have enough parallelism to keep the pipelines filled.
- You can also choose to pass the option `Bs=X` to control the thread block size for the GPU, where `X` is a power of 2 between 64 and 2048.

In [24]:
# cpu, Nh and T comes from grid search, peak: 1e12
make clean
make run_fma_prof Nh=1879048192 Nd=0 T=25 COPTFLAGS='-O3 -xHost' CUOPTFLAGS='-O3'

rm -f *.o *.optrpt *.so fma_prof fma_prof_opt
icc -g -Wall -std=c99 -fPIC -O3 -xHost -I/usr/local/pacerepov1/cuda/8.0.44/include -qopenmp -c -o fma_prof.o fma_prof.c
icc -g -Wall -std=c99 -fPIC -O3 -xHost -I/usr/local/pacerepov1/cuda/8.0.44/include -qopenmp -c -o fma_omp.o fma_omp.c
icc -g -Wall -std=c99 -fPIC -O3 -xHost -I/usr/local/pacerepov1/cuda/8.0.44/include -qopenmp -c -o fma_loop_host.o fma_loop_host.c
nvcc -ccbin=icpc -Xcompiler '-fPIC' -O3 -dc -o fma_cuda.o fma_cuda.cu
nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc -ccbin=icpc -Xcompiler '-fPIC' -O3 -dc -o fma_loop_dev.o fma_loop_dev.cu
nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc -ccbin=icpc -Xcompiler '-fPIC' -dlink  fma_cuda.o fma_loop_dev.o -o fma_cuda_

Nh and T comes from brute force grid search. peak flop in theory is ~1e12, we get 5e11 which is ~50%
one thing quite weird is that this number doesn't seem very reproducable, if I run with same parameters for 

In [26]:
> all_flops
for i in {23..24}
  do
    make clean
    echo $i
    make run_fma_prof Nh=256 Nd=0 T=25 COPTFLAGS='-O3 -xHost' CUOPTFLAGS='-O3' >> all_flops 2>&1
  done
grep -1 "fma_prof]" all_flops

rm -f *.o *.optrpt *.so fma_prof fma_prof_opt
23
rm -f *.o *.optrpt *.so fma_prof fma_prof_opt
24
OMP_PROC_BIND=spread OMP_NUM_THREADS=28  ./fma_prof 256 0 -1 -1 25 0.5 3.0
[./fma_prof] Nh = 256, Nd = 0, T = 25, default block size
[./fma_prof]: 1.096725e-05 elapsed seconds
[./fma_prof]: 12800 flops executed
[./fma_prof]: 1.167111e+09 flop/s
icc -g -Wall -std=c99 -fPIC -O3 -xHost -I/usr/local/pacerepov1/cuda/8.0.44/include -qopenmp -c -o fma_prof.o fma_prof.c
--
OMP_PROC_BIND=spread OMP_NUM_THREADS=28  ./fma_prof 256 0 -1 -1 25 0.5 3.0
[./fma_prof] Nh = 256, Nd = 0, T = 25, default block size
[./fma_prof]: 1.096725e-05 elapsed seconds
[./fma_prof]: 12800 flops executed
[./fma_prof]: 1.167111e+09 flop/s


In [16]:
make clean
make run_fma_prof
# baseline performance of the model: 1.3e11 flops/s

rm -f *.o *.optrpt *.so fma_prof fma_prof_opt
icc -g -Wall -std=c99 -fPIC -O3 -xHost -qopt-report=5 -I/usr/local/pacerepov1/cuda/8.0.44/include -qopenmp -c -o fma_prof.o fma_prof.c
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
icc -g -Wall -std=c99 -fPIC -O3 -xHost -qopt-report=5 -I/usr/local/pacerepov1/cuda/8.0.44/include -qopenmp -c -o fma_omp.o fma_omp.c
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
icc -g -Wall -std=c99 -fPIC -O3 -xHost -qopt-report=5 -I/usr/local/pacerepov1/cuda/8.0.44/include -qopenmp -c -o fma_loop_host.o fma_loop_host.c
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
nvcc -ccbin=icpc -Xcompiler '-fPIC'  -dc -o fma_cuda.o fma_cuda.cu
nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc -cc

**Compute Node Exercise 4 (Bonus 1 pt):** Now let's see if we can make any transformations to the code to make a difference.

We will run the same program, but with fused multiply add loops that you have tried to optimize.  You should edit the files
`fma_loop_host_opt.cu` and/or `fma_loop_dev_opt.c`: they start out exactly the same as the reference implementations used above.

In [ ]:
cat fma_loop_host_opt.c
# cat fma_loop_dev_opt.cu

In [ ]:
diff fma_loop_host.c fma_loop_host_opt.c
# diff fma_loop_dev.cu fma_loop_dev_opt.cu

See if you can exploit vectorization, instruction level parallelism, and/or loop transformations to get a boost.

In [35]:
make clean
make run_fma_prof_opt COPTFLAGS='-O3 -xHost' CUOPTFLAGS='-O3'
# compared with 1e11 above, manually vectorize with size 8 gives 1k times faster result(?!)
# then compute with a_i for more times gives higher effiency (this works similar as T)

rm -f *.o *.optrpt *.so fma_prof fma_prof_opt
icc -g -Wall -std=c99 -fPIC -O3 -xHost -I/usr/local/pacerepov1/cuda/8.0.44/include -qopenmp -c -o fma_prof.o fma_prof.c
icc -g -Wall -std=c99 -fPIC -O3 -xHost -I/usr/local/pacerepov1/cuda/8.0.44/include -qopenmp -c -o fma_omp.o fma_omp.c
icc -g -Wall -std=c99 -fPIC -O3 -xHost -I/usr/local/pacerepov1/cuda/8.0.44/include -qopenmp -c -o fma_loop_host.o fma_loop_host.c
nvcc -ccbin=icpc -Xcompiler '-fPIC' -O3 -dc -o fma_cuda.o fma_cuda.cu
nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc -ccbin=icpc -Xcompiler '-fPIC' -O3 -dc -o fma_loop_dev.o fma_loop_dev.cu
nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc -ccbin=icpc -Xcompiler '-fPIC' -dlink  fma_cuda.o fma_loop_dev.o -o fma_cuda_

In [37]:
make clean
make run_fma_prof_opt Nh=1879048192 Nd=0 T=1048576 COPTFLAGS='-O3 -xHost' CUOPTFLAGS='-O3' # modify this for peak flop/s
# given modified code above, increasing Nh gives even more efficiency (why?)

rm -f *.o *.optrpt *.so fma_prof fma_prof_opt
icc -g -Wall -std=c99 -fPIC -O3 -xHost -I/usr/local/pacerepov1/cuda/8.0.44/include -qopenmp -c -o fma_prof.o fma_prof.c
icc -g -Wall -std=c99 -fPIC -O3 -xHost -I/usr/local/pacerepov1/cuda/8.0.44/include -qopenmp -c -o fma_omp.o fma_omp.c
icc -g -Wall -std=c99 -fPIC -O3 -xHost -I/usr/local/pacerepov1/cuda/8.0.44/include -qopenmp -c -o fma_loop_host.o fma_loop_host.c
nvcc -ccbin=icpc -Xcompiler '-fPIC' -O3 -dc -o fma_cuda.o fma_cuda.cu
nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc -ccbin=icpc -Xcompiler '-fPIC' -O3 -dc -o fma_loop_dev.o fma_loop_dev.cu
nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc -ccbin=icpc -Xcompiler '-fPIC' -dlink  fma_cuda.o fma_loop_dev.o -o fma_cuda_

One thing I can not understand is that why my flop/s reach 1e16, is the peak flops calculation above is wrong, or some issue in the runtime report? or i am allocating resource wrong way?

## Submitting this work

**Workstation exercise 1 (1 pt):** When you have completed the rest of this assignment, `git add` the changes to this file, the source files you modified, and any scripts you added, and `git commit` them.  Having commited your changes, you should `git push` them to the private repository that you have on `github.gatech.edu`.

Our TA Han Sol Suh will email each of you a individualized [deploy key](https://developer.github.com/v3/guides/managing-deploy-keys/) that will allow him to read the contents of your repository.  

**Assignments need to be formally submitted to canvas,** but the totality of your submission on canvas should be a git revision hash or branch name indicating the version of your repository we should use to grade the assignment.